### Introduction

Now you have a clean deployment of the studio and it is time to start using it. The steps below will enable you to onboard some initial artefacts, before trying out the functionality.

Navigate to the UI front page and create an api key. Click on the Manage your API keys link. This should pop-up a window where you can generate, access and delete your api keys.


### Set up and  Installation 
#### Prerequisites

Create a pythin environment 
```bash
python -m venv venv
source venv/bin/activate
```
Install geostudio sdk
```bash
pip install geostudio

In [1]:
# Import the required packages
import json
import rasterio
import matplotlib.pyplot as plt

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from geostudio import Client
from geostudio import gswidgets

### Connecting to the platform
First, we set up the connection to the platform backend.  To do this we need the base url for the studio UI and an API key.

To get an API Key:
1. Go to your deployed version of the Geospatial Studio UI(https://localhost:4180/)page and navigate to the Manage your API keys link.
2.  This should pop-up a window where you can generate, access and delete your api keys. NB: every user is limited to a maximum of two activate api keys at any one time.

Store the API key and geostudio ui base url in a credentials file locally, for example in /User/bob/.geostudio_config_file. You can do this by:

```bash
echo "GEOSTUDIO_API_KEY=<paste_api_key_here>" > .geostudio_config_file
echo "BASE_STUDIO_UI_URL=<paste_ui_base_url_here>" >> .geostudio_config_file
```

Copy and paste the file path to this credentials file in call below.

In [ ]:
#############################################################
# Initialize Geostudio client using a geostudio config file
#############################################################
gfm_client = Client(geostudio_config_file=".geostudio_config_file")

In [18]:
# segmentation template

with open('payloads/templates/template-seg.json', 'r') as f:
    template_seg = json.load(f)
template_response=gfm_client.create_task(template_seg,output='df')
display(template_response)


,id
0,4679be6d-b652-4400-a51d-abc53aa62031


Load a complete tune to the studio

In [10]:
# Load a prithvi-eo-flood complete tune to the studio
with open('payloads/tunes/tune-prithvi-eo-flood.json','r') as f:
    complete_tune = json.load(f)
tune_response=gfm_client.upload_completed_tunes(complete_tune)
display(json.dumps(tune_response,indent=2))

'{\n  "message": "Upload started",\n  "tune_id": "geotune-4vrvuovdpkggdj4z8hh4py"\n}'

In [ ]:
gfm_client.poll_finetuning_until_finished(tune_response['tune_id'])

### Submit inference

Replace the tune id from the UI and move to finetuning and then to models and tune ,click on geofm-sandbox-models available for demonstration.Thos tune was loaded during the installation of studio and paste it below
Optionally you can use your own tune , jsut copy the tune id after uploading tune above is complete

In [15]:

# IMPORTANT: Replace with your tune_id from the UI
tune_id = "geotune-chy6wdpecsc8rbzrjkae5v"

# Define the inference payload
payload = {
    "model_display_name": "geofm-sandbox-models",
    "fine_tuning_id":tune_id,
    "location": "Dakhin Petbaha, Raha, Nagaon, Assam, India",
    "description": "Flood Assam local with sentinel aws",
    "spatial_domain": {
        "bbox": [
            [92.703396, 26.247896, 92.748087, 26.267903]
        ],
        "urls": [],
        "tiles": [],
        "polygons": []
    },
    "temporal_domain": [
        "2024-07-25_2024-07-28"
    ]
}

# Submit the inference request
gfm_client.try_out_tune(tune_id=tune_id, data=payload)


{'spatial_domain': {'bbox': [[92.703396, 26.247896, 92.748087, 26.267903]],
  'polygons': [],
  'tiles': [],
  'urls': []},
 'temporal_domain': ['2024-07-25_2024-07-28'],
 'fine_tuning_id': 'geotune-chy6wdpecsc8rbzrjkae5v',
 'generic_processor': None,
 'maxcc': 100,
 'model_display_name': 'geofm-sandbox-models',
 'description': 'Flood Assam local with sentinel aws',
 'location': 'Dakhin Petbaha, Raha, Nagaon, Assam, India',
 'geoserver_layers': None,
 'demo': None,
 'model_id': '5d47998b-4f7b-487e-93f2-25f3d7bc7e78',
 'inference_output': None,
 'generic_processor_id': None,
 'id': '6a01d836-f36a-4c4e-947a-63157aa61aea',
 'active': True,
 'created_by': 'test@example.com',
 'created_at': '2026-02-04T10:53:11.995841Z',
 'updated_at': '2026-02-04T10:53:12.009898Z',
 'status': 'PENDING',
 'tasks_count_total': 1,
 'tasks_count_success': 0,
 'tasks_count_failed': 0,
 'tasks_count_stopped': 0,
 'tasks_count_waiting': 1}

###  Onboard a Tuning Dataset from COS
On board burn scars data set to the Studio ,You can monitor this using the UI (Dataset factory) or polling below

In [ ]:
with open("payloads/datasets/dataset-burn_scars.json","r") as f:
    wild_fire_dataset= json.load(f)
onboard_dataset_response=gfm_client.onboard_dataset(data=wild_fire_dataset)
display(json.dumps(onboard_dataset_response,indent=2))

'{\n  "Dataset": "submitting - adding dataset and labels",\n  "dataset_id": "geodata-fevqgoyhbpvazcnkkkma4p",\n  "dataset_url": "https://s3.us-east.cloud-object-storage.appdomain.cloud/geospatial-studio-example-data/burn-scar-training-data.zip"\n}'

#### Monitor onboarding status

You can then monitor the status of the onboarding process through the API with the get_dataset() function or polling function. You can alternatively monitor progress and view the dataset in the UI.

In [ ]:
gfm_client.poll_onboard_dataset_until_finished(onboard_dataset_response["dataset_id"])

### Onboard Backbone Model

In [15]:
backbone={
    "name" : "Prithvi_EO_V2_300M",
    "description" : "Geospatial pre-traineed foundation model Prithvi-EO-V2 (Prithvi_EO_V2_300M)",
    "checkpoint_filename" : "prithvi_eo_v2_300\/Prithvi_EO_V2_300M.pt",
    "model_params" : {"backbone": "prithvi_eo_v2_300", "embed_dim": 768, "num_heads": 12, "tile_size": 1, "num_layers": 12, "patch_size": 16, "tubelet_size": 1, "model_category": "prithvi"}
}
onboard_backbone_response=gfm_client.create_base_model(backbone)
display( json.dumps(onboard_backbone_response,indent=2))

'{\n  "id": "99b0ecae-9f93-48c0-bd86-869f8748f7ba"\n}'

### Finetune
### Tuning a model from a dataset in a cluster deployments with accessible GPUs
Now we can trigger a fine tuning job, using the payload and script below. First replace the values of keys *dataset_id*, *base_model_id** and *tune_template_id* with the ids generated after onboarding 

NOTE:  Currently, for local deployments with access to non-NVIDIA GPUs (i.e. Mac), you will need to run the fine-tuning outside of the local cluster, and the resulting model can be onboarded back to the local cluster for inference.

In [ ]:
# Get  the dataset id, base model id and tune template id
dataset_id = onboard_dataset_response['dataset_id']
base_model_id = onboard_backbone_response['id']
tune_template_id = template_response['id']

In [ ]:
payload={
  "name": "burn-scars-demo",
  "description": "Segmentation",
  "dataset_id": dataset_id,
  "base_model_id": base_model_id,
  "tune_template_id": tune_template_id,
}

tune_submitted=gfm_client.submit_tune( payload,output='json')
print(tune_submitted)

{'tune_id': 'geotune-fcshwtqwq8sasyia2q8sei', 'mcad_id': 'kjob-geotune-fcshwtqwq8sasyia2q8sei', 'status': 'In_progress', 'message': None}


In [ ]:
gfm_client.poll_finetuning_until_finished(tune_id=tune_submitted['tune_id'])

### Mac GPU Tuning (Alternative Workflow)

For Mac users without NVIDIA GPUs, you can prepare the tuning configuration in the Studio and run it locally with TerraTorch.

### Prepare Tuning Configuration

In [ ]:
# Install terratorch and ml flow
! pip install terratorch
! pip install 'mlflow>=1.0.0'

In [ ]:
import requests
import os

# IMPORTANT: Replace these IDs with your actual IDs
dataset_id = "geodata-vecsd7x8fybxyfhazxqncq"
base_model_id = "66ca20bb-d636-4b78-a27f-e8b15dfb701f"
tune_template_id = "f4f3a9ad-0afe-4de5-9952-a99f3cecd7e8"

payload = {
    "name": "burn-scars-demo",
    "description": "Segmentation",
    "dataset_id": dataset_id,
    "base_model_id": base_model_id,
    "tune_template_id": tune_template_id,
    "model_parameters": {
        "runner": {"max_epochs": "10"},
        "optimizer": {"lr": 6e-05, "type": "AdamW"}
    }
}
# gfm_client.submit_tune(payload)

# Get dry-run config
url = f"{os.environ['UI_ROUTE_URL']}/studio-gateway/v2/submit-tune/dry-run"
headers = {
    "Content-Type": "application/json",
    "X-API-Key": os.environ['STUDIO_API_KEY']
}

response = requests.post(url, json=payload, headers=headers, verify=False)

# Save config to file
with open('wildfire_prthvi.yaml', 'w') as f:
    f.write(response.text)

# Display the config
print(response.text)


This script converts a Kubernetes/cluster configuration to a local development configuration by:

Service URLs → Local URLs

MLflow tracking server: cluster service → localhost


Cluster storage paths → Local filesystem paths

Kubernetes PVCs (Persistent Volume Claims) → local directories

In [ ]:
%%bash
../deployment-scripts/localize_config.sh  wildfire_prithvi.yaml

In [ ]:
%%bash
terratorch fit -c config.yaml